In [71]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ML_utils import load_data,load_data_test_set, total_day_eve_night_grouping
from sklearn.preprocessing import PolynomialFeatures
df =pd.read_csv('train.csv')

### UNCOMMENT THIS PART TO GROUP FEATURES
feature_engineering = True
df=total_day_eve_night_grouping(df,grouping=feature_engineering)
exclude_list = [feature_name for feature_name in df.columns if feature_name not in['number_vmail_messages','number_customer_service_calls','total_charges','churn']]
one_hot_on = True
normalize_on = True
oversample_on=True

###################
#### LOAD DATA ####

X_train,y_train,X_val,y_val,X_test,y_test,columns = load_data(df,exclude=exclude_list,one_hot=one_hot_on,normalize_=normalize_on,oversample=oversample_on)


y_train[y_train==0]=-1

poly = PolynomialFeatures(degree=2)

X_train = poly.fit_transform(X_train)

X_train=np.concatenate((X_train,np.ones((X_train.shape[0],1))),axis=1)


X_val = poly.fit_transform(X_val)



In [72]:
lr_list=[0.01,0.1,1]
C=100
epoch_list=[100,1000,10000]


In [73]:
from sklearn.metrics import accuracy_score

In [74]:
def cost(w,Z,lam):
    Z=Z.T
    u = w.T.dot(Z) # as in (23)
    return (np.sum(np.maximum(0, 1 - u)) + \
        .5*lam*np.sum(w*w)) - .5*lam*w[-1]*w[-1] # no bias 


def grad(w,X_train,y_train,lam=0.1):
    Z=y_train*X_train
    
    Z=Z.T
 
    u = w.T.dot(Z) # as in (23)
    H = np.where(u < 1)[1]
    ZS = Z[:, H]

    g = (-np.sum(ZS, axis = 1, keepdims = True) + lam*w)
    g[-1] -= lam*w[-1] # no weight decay on bias
    return g

eps = 1e-4

def grad_descent(w0, X_train, y_train,lr=0.01,lam=0.1):
    best_accuracy = -1

    w = w0
    it = 0 
    while it < 100000:
        it = it + 1
        g = grad(w,X_train,y_train,lam)
        w -= lr*g
        y_pred=predict(w,X_val)
       
        if (it % 10000) == 1:
            Z=y_train*X_train
            print('it =', it, ', cost =', cost(w,Z,lam)[0])
        if np.linalg.norm(g) < 1e-5:
            break 
    return w

def predict(w,X):
    w=w[:-1]
    b=w[-1]
    Z=X.dot(w)+b
    Z=np.sign(Z)
    Z[Z==-1]=0
    return Z


In [75]:
w0=np.random.randn(X_train.shape[1],1)

w=grad_descent(w0,X_train,y_train,lr=0.01,lam=1./1)

it = 1 , cost = 37160.754151263616
it = 10001 , cost = 9887.419359945967
it = 20001 , cost = 7770.5591109222
it = 30001 , cost = 6484.983241182017
it = 40001 , cost = 13067.531598675352
it = 50001 , cost = 10453.625862226501
it = 60001 , cost = 96039.91336395474
it = 70001 , cost = 8070.748429392319
it = 80001 , cost = 12303.909311296871
it = 90001 , cost = 36436.06836231387


In [76]:
y_pred=predict(w,X_val)


In [77]:
y_pred.shape

(637, 1)

In [80]:
from sklearn.metrics import classification_report,accuracy_score
print(classification_report(y_val,y_pred))

              precision    recall  f1-score   support

         0.0       0.97      0.07      0.13       537
         1.0       0.17      0.99      0.28       100

    accuracy                           0.21       637
   macro avg       0.57      0.53      0.20       637
weighted avg       0.85      0.21      0.15       637



In [79]:
### USE THE MODEL ON THE TEST SET
df_test=pd.read_csv('test.csv')

df_test=total_day_eve_night_grouping(df_test,grouping=feature_engineering)


X_test_output=load_data_test_set(df_test,exclude=exclude_list+['id'],one_hot=one_hot_on,normalize_=normalize_on)
X_test_output = poly.fit_transform(X_test_output)
y_pred_output=predict(w,X_test_output)
y_pred=np.sign(y_pred)
y_pred[y_pred==-1]=0

y_pred_output=np.where(y_pred_output>0.5,'yes', 'no')
y_pred_output=y_pred_output.reshape(-1,)
id_column = np.arange(1, y_pred_output.shape[0] + 1)


# Create a DataFrame
df_output = pd.DataFrame({
    'id': id_column,
    'churn': y_pred_output
})


# Save the DataFrame as a CSV file
df_output.to_csv('output_svm_GD.csv', index=False)